# Data Prep of the CA NFIP claims, from FEMA
##  Naomi Kalman
##  Last edited: 1/4/2023

This code loads the raw json file and cleans the data up to the point of making dummy variables

Version 3 is revised and rearranged to match the flow of the final paper and make the code look nice for pasting. Any intermediate steps and thought processes have been removed, check in draft 2 folder for 02_data_prep2.ipynb for the working-draft code, especially the regex example for updating the flood zone.


# Goals: 
- shortest path to clean data
- properly label categorical fields
- run VIF
- create dummy variables
- export the clean data so can run the model without making the dataset over and over

# 1. Load the data

In [1]:
#load packages to read the json
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

#code from Intermediate Importing Data in Python, Hugo Bowne-Anderson, Ch. 2: Interacting with APIs to import Data from the web, Video 1: Intro to APIs and JSON

In [3]:
#load packages to read the json
import pandas as pd
import json

#load the json file
with open('output3.json','r') as json_file:
    data = json.load(json_file)

#create a dataframe from the key
df = pd.DataFrame(data['FimaNfipClaims'])

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:

df.duplicated().sum()

df.info()


In [ ]:
df.info()

# 2. Describe the Data

In [ ]:
#create a dataframe of the null value counts, name the column "count"
df_null = pd.DataFrame(df.isna().sum(), columns=['count'])

#add a column to calculate the percentage of null values
df_null['pct_null'] = round(df_null['count']/df.shape[0],4)*100

In [ ]:
print(df_null)

# 3. Preprocessing

Note that the order matters here, because as you remove columns and records the remaining values and categories change. Do the key indicators first. Dropping the non-residential properties first makes the claim amount part easier because there are fewer outliers.

Drop columns about geographic info such as lat/lon, zip code

Drop columns that are poorly populated, drop rows where key metrics are sparse, then see what else is missing.

In [4]:
#drop geo columns
geo_cols = ['reportedCity','countyCode','censusTract','latitude','longitude','reportedZipcode','state']
df.drop(columns=geo_cols, inplace=True)

#drop sparse columns 
sparse_cols = ['baseFloodElevation','basementEnclosureCrawlspace','elevationCertificateIndicator',\
    'elevationDifference','lowestAdjacentGrade','amountPaidOnIncreasedCostOfComplianceClaim',\
    'obstructionType','communityRatingSystemDiscount']
df.drop(columns=sparse_cols, inplace=True)

#drop date columns
date_cols = ['asOfDate','originalNBDate','dateOfLoss','yearOfLoss','originalConstructionDate']
df.drop(columns=date_cols, inplace=True)

#drop non-relevant columns
other_cols =['lowestFloorElevation','id','agricultureStructureIndicator','houseWorship',\
    'nonProfitIndicator','rateMethod','smallBusinessIndicatorBuilding']
df.drop(columns=other_cols, inplace=True)

In [5]:
df.columns

Index(['condominiumIndicator', 'policyCount', 'elevatedBuildingIndicator',
       'floodZone', 'locationOfContents', 'numberOfFloorsInTheInsuredBuilding',
       'occupancyType', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim', 'postFIRMConstructionIndicator',
       'totalBuildingInsuranceCoverage', 'totalContentsInsuranceCoverage',
       'primaryResidence'],
      dtype='object')

## remove non-residential properties
Values are 4, 17, 18 and 19. see metadata at: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v1

Values are actually anything not 1 or 2 because everything else has different limits: 
https://www.fema.gov/sites/default/files/documents/fema_fim-3-how-to-write_apr2021.pdf

In [6]:
#keep only the residential records and display the value counts
#create a list of the residential occupancy codes
my_list = [1.0,2.0]
df = df.loc[df['occupancyType'].isin(my_list)]

#replace the numeric codes with their text values
occ_type_dict = {1.0:'single family',2.0:'2 to 4 unit bldg'}
df['occupancyType'].replace(occ_type_dict,inplace=True)

print(df['occupancyType'].value_counts())
print('Total remaining records:', df.shape[0])

single family       40071
2 to 4 unit bldg     3334
Name: occupancyType, dtype: int64
Total remaining records: 43405


## handle null values and zero-dollar claims in the buildings and contents

Note: because null + number = null, need to fill the null values as zeros for the claim and insured amount columns. Then can do other processing, like divide by the number of policies, or add the two together

Building and contents policies are taken out separately, so add total coverage together too

### impute the missing values

In [7]:
# fill the null values in building and contents claims with the number 0
df['amountPaidOnBuildingClaim'] = df['amountPaidOnBuildingClaim'].fillna(0)
df['amountPaidOnContentsClaim'] = df['amountPaidOnContentsClaim'].fillna(0)

### calculate the total damage and coverage
### Drop records where total claim = 0; there should be a claim payment
### remove contents over 100k, buildings over 250k

In [8]:
#calculate the sum of the building and contents payment
df['dmg_total'] = df['amountPaidOnBuildingClaim'] + df['amountPaidOnContentsClaim']

#drop the records where total claim = 0
df = df.loc[df['dmg_total']>0]

#remove the claims over their individual maximums
df = df.loc[df['amountPaidOnBuildingClaim']<=250000]
df = df.loc[df['amountPaidOnContentsClaim']<=100000]

print('Records over $350,000 total: ' ,df.loc[df['dmg_total']>350000].shape[0])
print('Total remaining records:', df.shape[0])

Records over $350,000 total:  0
Total remaining records: 28361


### repeat for the coverage

In [9]:
# fill the null values in building and contents coverage with the number 0
df['totalBuildingInsuranceCoverage'] = df['totalBuildingInsuranceCoverage'].fillna(0)
df['totalContentsInsuranceCoverage'] = df['totalContentsInsuranceCoverage'].fillna(0)

#calculate the sum of the building and contents coverage
df['cov_total'] = df['totalBuildingInsuranceCoverage'] + df['totalContentsInsuranceCoverage']

## drop records with policy count greater than 1

In [10]:
#drop records with multiple policies
df = df.loc[df['policyCount']==1]
print('Total remaining records:', df.shape[0])

Total remaining records: 28220


## Flood zone: drop any null values

note that even though it is shown second in the write-up, it has to be done first because you cannot mask with NaN values (.loc[] uses a mask)

In [11]:
#drop null flood zone
df.dropna(subset=['floodZone'],inplace=True)
print('Total remaining records:', df.shape[0])

Total remaining records: 24748


## Flood zone: group/rename the zones

Plan: anything that starts with A should be A, anything B or X-Shaded should be B, C or X = C. D is its own category, maybe combine with other, unknown, or excluded. Decide about V, same hazard level as A but different action

use this glossary to map the flood zones
https://www.fema.gov/glossary/flood-zones

after getting the letters sorted, try low (X,C), medium (B,X shaded), high(any A, any V). Not sure where to put D because it is "unknown" but needs to be in the list
https://www.fema.gov/sites/default/files/2020-08/fema_understanding-zone-D-levees.pdf

Wow, they don't actually differentiate between X and X shaded, since those are in two different columns in their dataset. They have X listed in the metadata as either B, C or X. So their own ridiculous data structure has ruined this attribute. I am going to have to treat X as its own thing. There are also no dual flood zones listed. 

In [12]:
#create a new column called flood group
df['flood_group'] = df['floodZone']

#update the values that start with A
df.loc[df['floodZone'].str.contains('A'), 'flood_group'] = 'A'

#update the values that start with v
df.loc[df['floodZone'].str.contains('V'), 'flood_group'] = 'V'

#show results
df['flood_group'].value_counts()

A    15420
X     4138
C     3330
B     1229
V      462
D      169
Name: flood_group, dtype: int64

## check data sparsity and drop null values from remaining non-categorical fields

In [ ]:
df.isna().sum()

In [ ]:
# #skip this, condo indicator is categorical
# #drop records with null values in remaining columns
# df.dropna(subset=['condominiumIndicator'],inplace=True)
# print('Total remaining records:', df.shape[0])

# Process categorical fields

## check the distribution of the values in all the categorical fields, univariate statistics

this part isn't used in the final because I did each field individually, but it is usefull to have the list and see which are going to need what

In [ ]:
cat_cols = ['condominiumIndicator','elevatedBuildingIndicator','locationOfContents'\
    ,'numberOfFloorsInTheInsuredBuilding','occupancyType','postFIRMConstructionIndicator'\
    ,'primaryResidence','flood_group']

In [ ]:
for col in cat_cols:
    print(df[col].value_counts().sort_index())

## this is all rearranged for the document flow of my paper

CONDO INDICATOR

In [13]:
#inspect the value counts for condo indicator
df['condominiumIndicator'].value_counts().sort_index()

A       13
L        1
N    24525
U      170
Name: condominiumIndicator, dtype: int64

In [14]:
#drop records from categories with fewer than 100 counts
bad_condo = ['L','A']
df = df.loc[~df['condominiumIndicator'].isin(bad_condo)]
print('Total remaining records:', df.shape[0])

Total remaining records: 24734


ELEVATED BUILDING

In [15]:
#inspect the value counts for elevated building indicator
df['elevatedBuildingIndicator'].value_counts().sort_index()

False    20852
True      3882
Name: elevatedBuildingIndicator, dtype: int64

In [16]:
#convert true/false to 1/0
df['elevatedBuildingIndicator'].replace({False:0,True:1}, inplace=True)

LOCATION OF CONTENTS

In [17]:
#inspect the value counts for location of contents field
df['locationOfContents'].value_counts().sort_index()

0    9362
1       2
2    1582
3    9070
4    4237
5      43
6     141
7     297
Name: locationOfContents, dtype: int64

In [18]:
#remove categories of locations with sparse entries
bad_loc = ['1','5']
df = df.loc[~df['locationOfContents'].isin(bad_loc)]
print('Total remaining records:', df.shape[0])

Total remaining records: 24689


In [19]:
#map the location of contents to descriptive text
#create a dictionary
loc_cont_dict = {0:np.NaN, 1:'Subgrade only', 2:'Subgrade and above',\
    3:'Lowest floor only',4:'Lowest floor and above',\
    5:'More than one floor above',6:'Mobile home',7:'Enclosure and above'}

#replace the values using the dictionary
df['locationOfContents'].replace(loc_cont_dict, inplace=True)

#show the value counts again
df['locationOfContents'].value_counts()

Lowest floor only         9070
Lowest floor and above    4237
Subgrade and above        1582
Enclosure and above        297
Mobile home                141
Name: locationOfContents, dtype: int64

NUMBER OF FLOORS

In [20]:
# replace the codes with text for number of floors
floors_dict = {1.0:'one',2.0:'two',3.0:'three or more',4.0:'split level',\
    5.0:'mobile home',6.0:'townhome'}
df['numberOfFloorsInTheInsuredBuilding'].replace(floors_dict, inplace=True)

#inspect the value counts for number of floors field
df['numberOfFloorsInTheInsuredBuilding'].value_counts()

one              14534
two               7819
three or more     1730
split level        441
mobile home        164
townhome             1
Name: numberOfFloorsInTheInsuredBuilding, dtype: int64

In [21]:
#remove the townhome category
df = df.loc[df['numberOfFloorsInTheInsuredBuilding']!='townhome']
print('Total remaining records:', df.shape[0])

Total remaining records: 24688


POST-FIRM INDICATOR

In [22]:
#convert true/false to 1/0
df['postFIRMConstructionIndicator'].replace({False:0,True:1}, inplace=True)

#inspect the value counts for the post-FIRM construction indicator
df['postFIRMConstructionIndicator'].value_counts()

0    21631
1     3057
Name: postFIRMConstructionIndicator, dtype: int64

PRIMARY RESIDENCE

In [23]:
#convert true/false to 1/0
df['primaryResidence'].replace({False:0,True:1}, inplace=True)

#inspect the value counts for the primary residence indicator
df['primaryResidence'].value_counts()

0    15497
1     9191
Name: primaryResidence, dtype: int64

## check data sparsity

In [24]:
#check data sparsity in the predictor variables and overall dataset
#predictor variables
X = df.loc[:,['condominiumIndicator','elevatedBuildingIndicator','locationOfContents'\
    ,'numberOfFloorsInTheInsuredBuilding','occupancyType','postFIRMConstructionIndicator'\
    ,'primaryResidence','flood_group','cov_total']]

print("total remaining records:", X.shape[0])
print("sparsity by column:\n", X.isna().sum())
#total sparsity is the sum of the sum of the null values in each column
#divided by the total size of the matrix
print("total data sparsity: ", X.isna().sum().sum()/(X.shape[0]*X.shape[1]))

total remaining records: 24688
sparsity by column:
 condominiumIndicator                    38
elevatedBuildingIndicator                0
locationOfContents                    9361
numberOfFloorsInTheInsuredBuilding       0
occupancyType                            0
postFIRMConstructionIndicator            0
primaryResidence                         0
flood_group                              0
cov_total                                0
dtype: int64
total data sparsity:  0.04230125297040398


## univariate stats

In [ ]:
import matplotlib.pyplot as plt

#univariate statistics, continuous variables
plt.hist(df['dmg_total'])
plt.title("Total Payment")
plt.xlabel("Dollars of Payment")
plt.ylabel("Number of Claims")
plt.show()

In [ ]:
#univariate statistics, continuous variables
plt.hist(df['cov_total'])
plt.title("Total Coverage")
plt.xlabel("Dollars of Coverage")
plt.ylabel("Number of Claims")
plt.show()

In [ ]:
# #histogram of total damage
# df['dmg_total'].hist()

In [ ]:
cat_cols = ['condominiumIndicator','elevatedBuildingIndicator','locationOfContents'\
    ,'numberOfFloorsInTheInsuredBuilding','occupancyType','postFIRMConstructionIndicator'\
    ,'primaryResidence','flood_group']

In [ ]:
# for col in cat_cols:
#     #plt.bar(df[col].unique(),df[col].value_counts())
#     plt.bar(df[col].value_counts().sort_index().index, df[col].value_counts().sort_index())
#     plt.title(col)
#     plt.show()

In [ ]:
#univariate statistics
from seaborn import sns

for col in cat_cols:
    sns.countplot(y=df[col], data=df)
    plt.show()

## bivariate stats

In [ ]:
#bivariate plot for continuous variables
sns.scatterplot(x=df['cov_total'],y=df['dmg_total'],data=df)

In [ ]:
for col in cat_cols:
    sns.boxplot(x=col,y='dmg_total',data=df)
    plt.show()

In [ ]:
y = df['amountPaidOnBuildingClaim']
x = df['amountPaidOnContentsClaim']

plt.scatter(x,y)
plt.xlabel("Amt Paid Contents")
plt.ylabel("Amt Paid Structure")
plt.show()

In [ ]:
y = df['dmg_total']
x = df['cov_total']

plt.scatter(x,y)
plt.show()

In [25]:
#Simplify the columns names
new_col_names = {'condominiumIndicator':'condo','elevatedBuildingIndicator':'elevated'\
    ,'numberOfFloorsInTheInsuredBuilding':'num_floors','postFIRMConstructionIndicator':'post_firm'\
    ,'amountPaidOnBuildingClaim':'dmg_bldg','amountPaidOnContentsClaim':'dmg_contents'\
    ,'totalBuildingInsuranceCoverage':'cov_bldg','totalContentsInsuranceCoverage':'cov_contents'}

df.rename(columns=new_col_names, inplace=True)

In [26]:
df.head()

,condo,policyCount,elevated,floodZone,locationOfContents,num_floors,occupancyType,dmg_bldg,dmg_contents,post_firm,cov_bldg,cov_contents,primaryResidence,dmg_total,cov_total,flood_group
0,N,1,0,X,NaN,two,single family,2260.90,0.00,0,185000,60000,0,2260.90,245000,X
1,N,1,0,X,NaN,one,single family,12183.13,0.00,0,250000,100000,1,12183.13,350000,X
2,N,1,0,AO,NaN,one,single family,42858.09,32886.55,0,203500,63000,1,75744.64,266500,A
4,N,1,0,X,Lowest floor and above,two,single family,9765.99,0.00,0,250000,100000,1,9765.99,350000,X
6,N,1,0,AE,NaN,one,single family,52103.49,30000.00,0,173700,30000,1,82103.49,203700,A


In [ ]:
#export to .csv to use for analysis
df.to_csv('df_clean.csv')

# INSTEAD OF TRYING TO MAKE ONE PERFECT DATASET AND MAKE THE DUMMY VARIABLES, EXPORT THE CLEANED UP THING AND MAKE THE DUMMIES IN THE NEXT STEP, THE ANALYSIS NOTEBOOK.
# CAN ALSO DO THE CATEGORIES IN THE NEXT STEP, BUT NOT GOING TO USE THAT IDEA FOR THE PAPER SO JUST LEAVING IT HERE FOR POSTERITY OR SOMETHING

## Add a column for in flood zone or out of flood zone, for another pass through the model, and same for location of contents

In [27]:
X_and_y = df.copy()

In [28]:
#flood zone
#write a dictionary to map the column
flood_bin_dict = {'A':'IN','B':'OUT','C':'OUT','D':'UNK','X':'OUT','V':'IN'}

#add a new column and assign the mapped values
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html
X_and_y=X_and_y.assign(flood_bin=X_and_y['flood_group'].map(flood_bin_dict))

X_and_y.head()


,condo,policyCount,elevated,floodZone,locationOfContents,num_floors,occupancyType,dmg_bldg,dmg_contents,post_firm,cov_bldg,cov_contents,primaryResidence,dmg_total,cov_total,flood_group,flood_bin
0,N,1,0,X,NaN,two,single family,2260.90,0.00,0,185000,60000,0,2260.90,245000,X,OUT
1,N,1,0,X,NaN,one,single family,12183.13,0.00,0,250000,100000,1,12183.13,350000,X,OUT
2,N,1,0,AO,NaN,one,single family,42858.09,32886.55,0,203500,63000,1,75744.64,266500,A,IN
4,N,1,0,X,Lowest floor and above,two,single family,9765.99,0.00,0,250000,100000,1,9765.99,350000,X,OUT
6,N,1,0,AE,NaN,one,single family,52103.49,30000.00,0,173700,30000,1,82103.49,203700,A,IN


In [29]:
#loc of contents
#dictionary
cont_bin_dict = {'Subgrade only':1,'Subgrade and above':1,'Lowest floor only':0,'Lowest floor and above':0,\
    'More than one floor above':0,'Mobile home':0,'Enclosure and above':0}

#add a new column and assign the mapped values
X_and_y=X_and_y.assign(cont_subgrade=X_and_y['locationOfContents'].map(cont_bin_dict))

X_and_y.head()

,condo,policyCount,elevated,floodZone,locationOfContents,num_floors,occupancyType,dmg_bldg,dmg_contents,post_firm,cov_bldg,cov_contents,primaryResidence,dmg_total,cov_total,flood_group,flood_bin,cont_subgrade
0,N,1,0,X,NaN,two,single family,2260.90,0.00,0,185000,60000,0,2260.90,245000,X,OUT,NaN
1,N,1,0,X,NaN,one,single family,12183.13,0.00,0,250000,100000,1,12183.13,350000,X,OUT,NaN
2,N,1,0,AO,NaN,one,single family,42858.09,32886.55,0,203500,63000,1,75744.64,266500,A,IN,NaN
4,N,1,0,X,Lowest floor and above,two,single family,9765.99,0.00,0,250000,100000,1,9765.99,350000,X,OUT,0.0
6,N,1,0,AE,NaN,one,single family,52103.49,30000.00,0,173700,30000,1,82103.49,203700,A,IN,NaN


In [ ]:
#export to .csv to use for next part
X_and_y.to_csv('X_and_y.csv')